In [1]:
import requests
import pandas as pd

import os
import os.path
import netCDF4 #as nc#not directly used but needs to be imported for some nc4 files manipulations, use for nc files
#from scipy.io import netcdf
from netCDF4 import Dataset
#import h5netcdf.legacyapi as netCDF4
import numpy as np
import numpy.ma as ma

import multiprocessing as mp# to download several file in parrallel
from multiprocessing import Pool

In [20]:
out_path=r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6'

In [ ]:
# count the number of CPU/cores
mp.cpu_count()

In [ ]:
NBCORES=5

In [32]:
# register information from csv file
#all_urls = pd.read_csv(r'C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\outputs\NEX-GDDP-CMIP6\gddp-cmip6-thredds-fileserver.csv')
csv_path = os.path.join(out_path,'gddp-cmip6-thredds-fileserver.csv')
all_urls = pd.read_csv(csv_path)

In [33]:
### make all elements of the csv into a readable list

# transpose csv
temp_list = all_urls[[' fileUrl']].T
temp_list=temp_list.values.tolist()
temp_list=temp_list[0]
url_list=[s.replace(' ', '') for s in temp_list]

In [34]:
## download only precipitation data
# select only precipitation files, between 2040 and 2080
url_list_precipitation = [url for url in url_list if 'pr_day_' in url and int(url[len(url)-7:len(url)-3])>2020 and int(url[len(url)-7:len(url)-3])<2061]

In [35]:
len(url_list_precipitation)

5160

In [ ]:
#### Former version
# download only precipitation files
r'''
for file in url_list_precipitation: # for loop to download the file in each url
    # find the name of the file
    index_before_name=file.rfind('/') # returns the highest index where the last character '/' was found, which is just before the name of the file    
    f_name = file[index_before_name+1:len(file)-3] # return the name of the file as a string
    # check if the file was already downloaded
    test = os.path.join('//COWI.net/projects/A245000/A248363/CRVA/Datasets/NEX-GDDP-CMIP6/',f_name+'.nc')
    if os.path.isfile(test): # if the file was aleready download
        continue # continue the for loop without executing the code after this line. The code follonwing download the file
        # if we entered the if, the file was already dowloaded, no need to downloaded it again
    # the file was not downloaded, the following code will permit to download it in the servor for dataset
    r = requests.get(file) # return the url were data need to be downloaded
    # download data in the servor for datasets
    with open(f'//COWI.net/projects/A245000/A248363/CRVA/Datasets/NEX-GDDP-CMIP6/{f_name}.nc', 'wb') as f:
        f.write(r.content)
'''

In [ ]:
def download_file(file):
    # find the name of the file
    index_before_name=file.rfind('/') # returns the highest index where the last character '/' was found, which is just before the name of the file    
    f_name = file[index_before_name+1:len(file)-3] # return the name of the file as a string
    # check if the file was already downloaded
    test = os.path.join('//COWI.net/projects/A245000/A248363/CRVA/Datasets/NEX-GDDP-CMIP6/',f_name+'.nc')
    if os.path.isfile(test): # if the file was aleready download
        #continue # continue the for loop without executing the code after this line. The code follonwing download the file
        return# if we entered the if, the file was already dowloaded, no need to downloaded it again
    # the file was not downloaded, the following code will permit to download it in the servor for dataset
    r = requests.get(file) # return the url were data need to be downloaded
    # download data in the servor for datasets
    with open(f'//COWI.net/projects/A245000/A248363/CRVA/Datasets/NEX-GDDP-CMIP6/{f_name}.nc', 'wb') as f:
        f.write(r.content)
    return file
results = ThreadPool(NBCORES).imap_unordered(download_file, url_list)
for r in results: #I don't understand why, ut without this step it does not work
     print(r)

In [ ]:
## read data, _FillValue should be NaN
solar_1950=Dataset(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc')
r'''
hurs=solar_1950.variables['hurs']
hurs
hurs._FillValue
solar_dataframe = np.ma.getdata(solar_1950.variables['hurs']).data
solar_dataframe
len(solar_dataframe[solar_dataframe!=1.e+20])
len(solar_dataframe[solar_dataframe==1.e+20])
type(solar_dataframe)
solar_dataframe.size
solar_dataframe.size-len(solar_dataframe[solar_dataframe!=1.e+20])-len(solar_dataframe[solar_dataframe==1.e+20])
'''

In [5]:
# some files only 1 KB
#name = 'pr_day_UKESM1-0-LL_ssp370_r1i1p1f2_gn_2052.nc'
#path_file = os.path.join(out_path,name)
pr_2052 = Dataset(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1951.nc')

In [ ]:
solar_dataframe=np.ma.getdata(pr_2052.variables['pr']).data
len(solar_dataframe[solar_dataframe!=1.e+20])

In [ ]:
len(solar_dataframe[solar_dataframe==1.e+20])

In [ ]:
netCDF4.Dataset(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\pr_day_UKESM1-0-LL_ssp370_r1i1p1f2_gn_2052.nc')

In [ ]:
os.path.isfile(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\pr_day_UKESM1-0-LL_ssp370_r1i1p1f2_gn_2052.nc')

In [ ]:
Dataset(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\pr_day_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_2060.nc', 'r', format='NETCDF4')

In [ ]:
os.path.isfile(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\pr_day_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_2060.nc')

In [29]:
## Hypothesis: some files are corrupted
# check if files are corrupted
#files = get_paths(source)  # A list of complete paths to each image
invalid_files = []
for name in os.listdir(out_path):
#name='hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1951.nc'
    try:
        path=os.path.join(out_path,name)
        im = Dataset(path)
        #print('Dataset')
    except (IOError, OSError):
        invalid_files.append(name)

In [30]:
len(invalid_files)

3514

In [31]:
invalid_files

['pr_day_MPI-ESM1-2-LR_ssp585_r1i1p1f1_gn_2048.nc',
 'pr_day_NorESM2-MM_ssp126_r1i1p1f1_gn_2036.nc',
 'pr_day_UKESM1-0-LL_ssp126_r1i1p1f2_gn_2060.nc',
 'pr_day_MIROC6_ssp370_r1i1p1f1_gn_2025.nc',
 'pr_day_NESM3_ssp245_r1i1p1f1_gn_2041.nc',
 'pr_day_MPI-ESM1-2-LR_ssp585_r1i1p1f1_gn_2035.nc',
 'pr_day_FGOALS-g3_ssp370_r3i1p1f1_gn_2046.nc',
 'pr_day_KIOST-ESM_ssp585_r1i1p1f1_gr1_2056.nc',
 'pr_day_CMCC-ESM2_ssp126_r1i1p1f1_gn_2042.nc',
 'pr_day_GFDL-ESM4_ssp126_r1i1p1f1_gr1_2049.nc',
 'pr_day_MIROC-ES2L_ssp126_r1i1p1f2_gn_2032.nc',
 'pr_day_TaiESM1_ssp585_r1i1p1f1_gn_2026.nc',
 'pr_day_CanESM5_ssp126_r1i1p1f1_gn_2039.nc',
 'pr_day_CNRM-CM6-1_ssp370_r1i1p1f2_gr_2052.nc',
 'pr_day_MPI-ESM1-2-LR_ssp126_r1i1p1f1_gn_2039.nc',
 'pr_day_CNRM-CM6-1_ssp126_r1i1p1f2_gr_2043.nc',
 'pr_day_IITM-ESM_ssp370_r1i1p1f1_gn_2025.nc',
 'pr_day_NESM3_ssp126_r1i1p1f1_gn_2036.nc',
 'pr_day_NorESM2-LM_ssp245_r1i1p1f1_gn_2047.nc',
 'pr_day_NorESM2-LM_ssp585_r1i1p1f1_gn_2034.nc',
 'pr_day_CESM2_ssp370_r4i1p1f1_gn_

In [40]:
# list url that should be downloaded again

url_corrupted_file = []
name_list_pr = []
for url in url_list_precipitation:
    index_before_name=url.rfind('/') # returns the highest index where the last character '/' was found, which is just before the name of the file    
    f_name = url[index_before_name+1:len(url)] # return the name of the file as a string
    name_list_pr.append(f_name)
    if f_name in invalid_files:
        url_corrupted_file.append(url)

In [39]:
len(url_corrupted_file)

3413

In [41]:
non_corrupted_file_pr = [name for name in name_list_pr if name not in invalid_files]

In [44]:
len(non_corrupted_file_pr)+len(url_corrupted_file)-len(url_list_precipitation)

0

In [46]:
path = os.path.join(out_path,non_corrupted_file_pr[0])
Dataset(path)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    activity: NEX-GDDP-CMIP6
    contact: Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget Thrasher: bridget@climateanalyticsgroup.org
    Conventions: CF-1.7
    creation_date: 2021-10-04T14:02:34.172359+00:00
    frequency: day
    institution: NASA Earth Exchange, NASA Ames Research Center, Moffett Field, CA 94035
    variant_label: r1i1p1f1
    product: output
    realm: atmos
    source: BCSD
    scenario: ssp245
    references: BCSD method: Thrasher et al., 2012, Hydrol. Earth Syst. Sci.,16, 3309-3314. Ref period obs: latest version of the Princeton Global Meteorological Forcings (http://hydrology.princeton.edu/data.php), based on Sheffield et al., 2006, J. Climate, 19 (13), 3088-3111.
    version: 1.0
    tracking_id: 6a0fbfc0-e8cb-4edd-9a4b-d720f6fbc956
    title: ACCESS-CM2, r1i1p1f1, ssp245, global downscaled CMIP6 climate projection data
    resolution_id: 0.25 degree
    history:

In [47]:
non_corrupted_file_pr

['pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2021.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2022.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2023.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2024.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2025.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2026.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2027.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2028.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2029.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2030.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2031.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2032.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2033.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2034.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2035.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2036.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2037.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2038.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2039.nc',
 'pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2040.nc',
 'pr_day_ACCESS-CM2_